<a href="https://colab.research.google.com/github/alesouzaeu/Coursera_Capstone/blob/main/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests2
!pip install beautifulsoup4

     |████████████████████████████████| 92kB 4.2MB/s 
     |████████████████████████████████| 61kB 4.8MB/s 
     |████████████████████████████████| 133kB 6.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: idna 2.10
    Uninstalling idna-2.10:
      Successfully uninstalled idna-2.10
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [98]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


In [99]:

# Chamamos a url que contém a tabela que queremos
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')
# Parseamos o Html desejado em variáveis para facilitar o trabalho
table = soup.find('table', {'class':'wikitable sortable'}).tbody

rows = table.find_all('tr')

columns = [v.text.replace('\n', '') for v in rows[0].find_all('th')]
# Definimos os nomes das colunas.
df_header =  pd.DataFrame(columns=columns)
# Criamos um loop para posicionar cada dado em sua respectiva coluna.
for i in range(1, len(rows)):
  tds = rows[i].find_all('td')

  if len(tds) ==4:
      values = [tds[0].text, tds[1].text, tds[2].text.replace('\n','')]# eliminamos caracteres indesejados por meio do método .replace
  else:
    values = [td.text.replace('\n','') for td in tds ]# eliminamos caracteres indesejados por meio do método .replace
 
  df = df.append(pd.Series(values, index=columns), ignore_index=True)




In [100]:
# Imprimimos o cabeçalho

df_header

,Postal Code,Borough,Neighbourhood


In [101]:
# Conferimos como ficou nosso dataframe

df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [102]:
# Extraímos este dataframe json.

df.to_json(r'list_of_postal_code_of_canada_m.json')

In [107]:
# Importamos o dataframe json.
df = pd.read_json('list_of_postal_code_of_canada_m.json')

In [106]:
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
1255,M5Z,Not assigned,Not assigned
1256,M6Z,Not assigned,Not assigned
1257,M7Z,Not assigned,Not assigned
1258,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [108]:
# Agora vamos descobrir a latitude e longitude de cada bairro pelo Postal Code